<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/computervision/semantic_sam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation

In [ ]:
!python -m pip install 'git+https://github.com/MaureenZOU/detectron2-xyz.git'
!pip install git+https://github.com/cocodataset/panopticapi.git
!git clone https://github.com/UX-Decoder/Semantic-SAM
%cd Semantic-SAM
!pip install -q timm einops transformers sentencepiece gradio kornia torchmetrics shapely

In [ ]:
# build encoder
%cd /content/Semantic-SAM/semantic_sam/body/encoder/ops/
!sh make.sh

Download mdoel

In [ ]:
# !https://github.com/UX-Decoder/Semantic-SAM/releases/download/checkpoint/swint_only_sam_many2many.pth -P /content/ckpt
!wget https://github.com/UX-Decoder/Semantic-SAM/releases/download/checkpoint/swinl_only_sam_many2many.pth -P /content/ckpt

Get demo Data

In [ ]:
!wget https://tshop.r10s.jp/unique-weddings/cabinet/uwppwhite/63/1.jpg -O /content/wedding_dress.jpg

Point Mask generation

In [ ]:
%cd /content/Semantic-SAM/

In [ ]:
from semantic_sam import prepare_image, plot_multi_results, build_semantic_sam, SemanticSAMPredictor
original_image, input_image = prepare_image(image_pth='/content/wedding_dress.jpg')  # change the image path to your image
mask_generator = SemanticSAMPredictor(build_semantic_sam(model_type='L', ckpt='/content/ckpt/swinl_only_sam_many2many.pth')) # model_type: 'L' / 'T', depends on your checkpint
iou_sort_masks, area_sort_masks = mask_generator.predict_masks(original_image, input_image, point=[[0.5, 0.5]]) # input point [[w, h]] relative location, i.e, [[0.5, 0.5]] is the center of the image
plot_multi_results(iou_sort_masks, area_sort_masks, original_image, save_path='../vis/')  # results and original images will be saved at save_path

Mask Autogenerator

In [ ]:
%cd /content/Semantic-SAM/

In [ ]:
from semantic_sam import prepare_image, plot_results, build_semantic_sam, SemanticSamAutomaticMaskGenerator
original_image, input_image = prepare_image(image_pth='/content/wedding_dress.jpg')  # change the image path to your image
mask_generator = SemanticSamAutomaticMaskGenerator(build_semantic_sam(model_type='L', ckpt='/content/ckpt/swinl_only_sam_many2many.pth'), level=[2]) # model_type: 'L' / 'T', depends on your checkpint
masks = mask_generator.generate(input_image)
plot_results(masks, original_image, save_path='../vis/')  # results and original images will be saved at save_path

In [ ]:
from tasks.interactive_idino_m2m_auto import show_anns
from PIL import Image
import matplotlib.pyplot as plt
fig = plt.figure()
plt.imshow(original_image)
show_anns([masks[3]])
fig.canvas.draw()
im = Image.frombytes('RGB', fig.canvas.get_width_height(), fig.canvas.tostring_rgb())
display(im)